In [ ]:
import model
import torch
import scanpy as sc
import anndata as ad
import dataloader as dl
from utils.get_edge_index import get_edge_index
from paste_ import get_pi

In [ ]:
data_path='/home/sxa/Datasets/Human_DLPFC/'
target_slice_id=151676
adjacent_slice_id=151675

In [ ]:
target_slice = dl.load_data(data_path,target_slice_id) 
adjacent_slice = dl.load_data(data_path,adjacent_slice_id) 

In [ ]:
target_slice_spots_num,starget_slice_genes_num = target_slice.X.shape[0],target_slice.X.shape[1]
adjacent_slice_spots_num,adjacent_slice_genes_num = adjacent_slice.X.shape[0],adjacent_slice.X.shape[1]

In [ ]:
slice1_slice2_pi = get_pi(target_slice, adjacent_slice)
slice1_slice2_pi_edge_index = get_edge_index(slice1_slice2_pi)

In [ ]:
hidden_dim = 1000
device = torch.device('cuda:0')

In [ ]:
model_ = model.model(target_slice_spots_num, adjacent_slice_spots_num, starget_slice_genes_num, adjacent_slice_genes_num, hidden_dim).to(device)

In [ ]:
# train
epochs = 1000
mse_loss = nn.MSELoss()

ae_optim = optim.Adam(model_.parameters(), lr=0.0001)
target_slice_X = torch.tensor(target_slice.X).to(device)
adjacent_slice_X = torch.tensor(adjacent_slice.X).to(device)

In [ ]:
for epoch in range(epochs):
    _, recreated_slice1_X = model_(slice1_slice2_pi_edge_index.to(device), target_slice_X, adjacent_slice_X) # slice1_edge_index.to(device)
    loss = mse_loss(recreated_slice1_X, target_slice_X)
    ae_optim.zero_grad()
    loss.backward()
    ae_optim.step()
    print("epoch=", epoch, " loss=", loss.data.float())